## Notebook Purpose

create, view different versions of poisson disc mask

In [1]:
import os, sys
import numpy as np
import torch
import matplotlib.pyplot as plt

sys.path.insert(0, '../')
from include.mri_helpers import generate_poisson_disc

### code for generating random masks

In [24]:
path_m = '/home/vanveen/ConvDecoder/masks/'

for accel in [4,8]:
    
    for c in [24, 32, 40, 64]:
        
        calib = (c,c)

        for ii in np.arange(20):

            mm = generate_poisson_disc(accel, calib=calib)

            fn_out = '{}mask_pd_{}x_calib{}_rand{}.npy'.format(
                                    path_m, accel, c, ii)
#             np.save(fn_out, mm)
            sys.exit()

given accel 4, actual accel 4.051000118255615


SystemExit: 

/home/vanveen/heck/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3425: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
